<a href="https://colab.research.google.com/github/rs845/COS429_Final_Project/blob/master/runExperiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# transfer data from drive to colab storage
zip_path = Path(COS429/FinalProject/archive.zip)
!cp “{zip_path}” .
!unzip -q archive.zip
!rm archive.zip

NameError: ignored

In [6]:
# import necessary packages
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import Xception # TensorFlow ONLY
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.densenet import preprocess_input as preprocess_densenet
from tensorflow.keras.applications.inception_v3 import preprocess_input as preprocess_inception
from tensorflow.keras.applications.xception import preprocess_input as preprocess_xception
from tensorflow.keras.applications.vgg19 import preprocess_input as preprocess_vgg19
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import argparse
import cv2

In [7]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive


In [ ]:
# !unzip -uq "/content/drive/MyDrive/COS429/FinalProject/DenseNet-BC-121-32-no-top.h5.zip" -d "/content/drive/MyDrive/COS429/FinalProject/DenseNet-BC-121-32-no-top.h5"

In [9]:
# import data
# importing data
base_path = 'COS429/FinalProject/archive/real_vs_fake/real-vs-fake/'
image_gen = ImageDataGenerator(rescale=1./255.)

train_flow = image_gen.flow_from_directory(
    base_path + 'train/',
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary'
)

valid_flow = image_gen.flow_from_directory(
    base_path + 'valid/',
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary'
)

test_flow = image_gen.flow_from_directory(
    base_path + 'test/',
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary'
)

Found 0 images belonging to 2 classes.
Found 2597 images belonging to 2 classes.
Found 10000 images belonging to 2 classes.


In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.version)
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

<module 'tensorflow._api.v2.version' from '/usr/local/lib/python3.6/dist-packages/tensorflow/_api/v2/version/__init__.py'>


SystemError: ignored

**Classification Models**

In [ ]:
def build_dense(pretrained):
    model = Sequential([
        pretrained,
        layers.GlobalAveragePooling2D(),
        layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(),
        metrics=['accuracy']
    )
    model.summary()
    return model

In [ ]:
def build_vgg(pretrained):
  model = Sequential([pretrained,
                    layers.Flatten(),
                    layers.Dense(2, activation = "softmax")])
  model.layers[0].trainable = False
  model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics="accuracy")
  model.summary()
  return model

In [ ]:
MODELS = {
    "denseNet" : DenseNet121,
    "inception" : InceptionV3,
    "xception" : Xception,
    "vgg" : VGG19
}

In [ ]:
training_steps = 100000//64
validation_steps = 20000//64

def DenseNet121():
  model = DenseNet121(
    weights= "/content/drive/MyDrive/COS429/FinalProject/DenseNet-BC-121-32-no-top.h5/DenseNet-BC-121-32-no-top.h5",
    include_top=False,
    input_shape=(224,224,3)
  ) 
  densenet = build_dense(model)
  densenet.summary()
  densenet.fit(
      train_flow,
      epochs = 1,
      steps_per_epoch = training_steps,
      validation_data = valid_flow,
      validation_steps = validation_steps
  )


def InceptionNetV3():
  model = InceptionV3(
      include_top=False,
      weights="imagenet",
      input_shape=(224,224,3),
      classifier_activation="softmax",
  )
  inceptionnet = build_dense(model)
  inceptionnet.summary()
  inceptionnet.fit(
      train_flow,
      epochs = 1,
      steps_per_epoch = 100000//64,
      validation_data = valid_flow,
      validation_steps = 20000//64
  )


def XceptionNet():
  model = Xception(
      include_top=False,
      weights="imagenet",
      input_shape=(224,224,3),
      classifier_activation="softmax",
  )
  Xceptionnet = build_model(model)
  Xceptionnet.summary()
  Xceptionnet.fit(
      train_flow,
      epochs = 1,
      steps_per_epoch = training_steps,
      validation_data = valid_flow,
      validation_steps = validation_steps
  )


def VGG():
  model = VGG19(
      include_top = False,
      weights="imagenet",
      input_shape=(224,224,3)
  )
  print(model.output[-1])
  VGGmodel = build_vgg(model)

  VGGmodel.fit(
      train_flow,
      epochs = 1,
      steps_per_epoch = training_steps,
      validation_data = valid_flow,
      validation_steps = validation_steps
  )



In [ ]:
DenseNet121()

In [ ]:
InceptionV3()

In [ ]:
VGG()